In [0]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [0]:
!pip install -U -q PyDrive
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials
import pandas as pd
# Authenticate and create the PyDrive client.
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)
link="https://drive.google.com/open?id=1uRkuyMI8G7tEtnArNwPl-JkVOtnj-uXT"
fluff, id = link.split('=')
downloaded = drive.CreateFile({'id':id})
downloaded.GetContentFile('500Cities_cleandata_ReadyForEDA.csv')
df = pd.read_csv('500Cities_cleandata_ReadyForEDA.csv')

In [0]:
import pandas as pd
import numpy as np
from sklearn.utils import shuffle
from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.naive_bayes import GaussianNB
from sklearn.svm import SVC
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.ensemble import BaggingClassifier

In [0]:
#Lets read in the data
df = pd.read_csv('/content/drive/My Drive/Colab Notebooks/Exam/500Cities_cleandata_ReadyForEDA.csv')

In [0]:
df.head()

,Unnamed: 0,StateAbbr,PlaceName,PlaceFIPS,Population2010,ACCESS2_CrudePrev,ARTHRITIS_CrudePrev,BINGE_CrudePrev,BPHIGH_CrudePrev,BPMED_CrudePrev,CANCER_CrudePrev,CASTHMA_CrudePrev,CHD_CrudePrev,CHECKUP_CrudePrev,CHOLSCREEN_CrudePrev,COLON_SCREEN_CrudePrev,COPD_CrudePrev,COREM_CrudePrev,COREW_CrudePrev,CSMOKING_CrudePrev,DENTAL_CrudePrev,DIABETES_CrudePrev,HIGHCHOL_CrudePrev,KIDNEY_CrudePrev,LPA_CrudePrev,MAMMOUSE_CrudePrev,MHLTH_CrudePrev,OBESITY_CrudePrev,PAPTEST_CrudePrev,PHLTH_CrudePrev,SLEEP_CrudePrev,STROKE_CrudePrev,TEETHLOST_CrudePrev,Geolocation
0,1,CA,Folsom,624638,72203,7.5,16.9,21.8,25.7,64.8,5.8,8.6,4.1,64.7,78.1,76.6,4.1,37.1,33.3,12.2,74.7,6.7,29.1,2.1,14.3,80.4,9.9,23.8,84.3,8.9,33.9,1.9,6.8,"(38.67504943280, -121.147605753)"
1,2,FL,Largo,1239425,77648,19.6,30.6,16.9,36.1,81.0,9.0,7.9,9.8,77.5,80.2,64.6,10.0,33.7,33.2,20.7,58.6,12.1,39.0,3.7,31.0,75.7,13.1,28.3,77.1,15.4,37.7,4.5,18.3,"(27.90909077340, -82.7714203383)"
2,3,CA,Berkeley,606000,112580,7.7,15.1,19.6,20.9,68.2,4.9,8.8,3.7,64.7,70.0,75.4,3.7,38.2,36.6,11.2,70.0,6.5,27.1,2.1,14.2,81.5,10.9,18.5,83.2,8.2,32.2,1.9,6.7,"(37.87256787650, -122.274907975)"
3,4,CA,Napa,650258,76915,12.3,20.7,19.2,28.1,70.2,6.5,8.9,5.8,63.8,75.4,69.3,5.9,37.9,30.3,14.5,70.2,8.9,34.1,2.8,19.8,76.7,12.0,24.0,83.9,12.0,32.7,2.8,11.2,"(38.29804246490, -122.301093331)"
4,5,FL,Sunrise,1269700,84439,22.8,22.8,16.3,33.3,76.7,6.5,8.0,6.7,77.7,78.7,59.7,7.0,30.5,26.2,16.5,61.0,12.1,37.1,3.2,29.5,82.5,12.7,28.1,81.3,13.3,38.1,3.7,16.2,"(26.15468783030, -80.2998411020)"


In [0]:
df.shape

(500, 34)

In [0]:
#Lets set the random seed to 123
np.random.seed(123)

In [0]:
#Time to shuffle our data
df = shuffle(df)

In [0]:
#Lets calculate mean of Population2010 first
median_pop = round(np.median(df['Population2010']),2)

In [0]:
print(median_pop)

106106.0


In [0]:
#Lets recode columns now based on median value
df['Population2010'] = np.where(df['Population2010'] > median_pop, '1', '0')

In [0]:
df.head()

,Unnamed: 0,StateAbbr,PlaceName,PlaceFIPS,Population2010,ACCESS2_CrudePrev,ARTHRITIS_CrudePrev,BINGE_CrudePrev,BPHIGH_CrudePrev,BPMED_CrudePrev,CANCER_CrudePrev,CASTHMA_CrudePrev,CHD_CrudePrev,CHECKUP_CrudePrev,CHOLSCREEN_CrudePrev,COLON_SCREEN_CrudePrev,COPD_CrudePrev,COREM_CrudePrev,COREW_CrudePrev,CSMOKING_CrudePrev,DENTAL_CrudePrev,DIABETES_CrudePrev,HIGHCHOL_CrudePrev,KIDNEY_CrudePrev,LPA_CrudePrev,MAMMOUSE_CrudePrev,MHLTH_CrudePrev,OBESITY_CrudePrev,PAPTEST_CrudePrev,PHLTH_CrudePrev,SLEEP_CrudePrev,STROKE_CrudePrev,TEETHLOST_CrudePrev,Geolocation
229,230,TX,Longview,4843888,0,22.3,25.3,17.3,34.5,77.2,6.4,9.7,6.8,69.8,71.9,60.8,7.2,35.2,29.5,19.8,55.8,12.2,37.7,3.2,28.8,75.3,13.6,36.2,80.8,13.8,35.2,3.7,16.5,"(32.51928578560, -94.7621975980)"
337,338,NV,North Las Vegas,3251800,1,22.4,20.2,16.5,25.7,68.3,4.5,9.1,5.5,67.2,67.1,53.8,6.7,29.6,25.0,21.1,54.2,11.3,32.5,2.9,27.8,74.4,15.8,29.0,79.8,14.4,43.1,3.0,18.2,"(36.28919935620, -115.088496088)"
327,328,MI,Ann Arbor,2603000,1,9.0,18.3,23.0,19.1,68.1,4.6,9.8,3.6,66.1,66.4,73.4,4.4,38.6,41.2,13.2,75.5,5.7,29.3,1.9,15.7,84.1,12.3,23.4,83.6,7.9,31.1,1.7,6.6,"(42.27568335670, -83.7311374448)"
416,417,CO,Westminster,883835,1,11.3,20.7,20.7,22.5,66.9,5.5,8.9,4.2,60.6,74.6,65.2,4.3,37.9,34.9,17.2,68.7,6.1,30.6,2.2,15.4,74.5,10.6,24.0,83.5,9.7,29.1,2.1,11.0,"(39.88385114750, -105.062645316)"
306,307,TX,Corpus Christi,4817000,1,27.4,23.9,18.1,32.5,76.7,5.4,8.6,6.5,69.8,72.3,60.4,6.0,31.5,29.8,16.7,56.9,14.4,38.2,3.2,30.0,79.8,12.2,37.2,82.3,14.1,33.9,3.1,15.1,"(27.72915883370, -97.4019095379)"


In [0]:
#Lets drop all the rows with NA values
df = df.dropna()

In [0]:
df.shape

(453, 34)

In [0]:
df.head()

,Unnamed: 0,StateAbbr,PlaceName,PlaceFIPS,Population2010,ACCESS2_CrudePrev,ARTHRITIS_CrudePrev,BINGE_CrudePrev,BPHIGH_CrudePrev,BPMED_CrudePrev,CANCER_CrudePrev,CASTHMA_CrudePrev,CHD_CrudePrev,CHECKUP_CrudePrev,CHOLSCREEN_CrudePrev,COLON_SCREEN_CrudePrev,COPD_CrudePrev,COREM_CrudePrev,COREW_CrudePrev,CSMOKING_CrudePrev,DENTAL_CrudePrev,DIABETES_CrudePrev,HIGHCHOL_CrudePrev,KIDNEY_CrudePrev,LPA_CrudePrev,MAMMOUSE_CrudePrev,MHLTH_CrudePrev,OBESITY_CrudePrev,PAPTEST_CrudePrev,PHLTH_CrudePrev,SLEEP_CrudePrev,STROKE_CrudePrev,TEETHLOST_CrudePrev,Geolocation
229,230,TX,Longview,4843888,0,22.3,25.3,17.3,34.5,77.2,6.4,9.7,6.8,69.8,71.9,60.8,7.2,35.2,29.5,19.8,55.8,12.2,37.7,3.2,28.8,75.3,13.6,36.2,80.8,13.8,35.2,3.7,16.5,"(32.51928578560, -94.7621975980)"
337,338,NV,North Las Vegas,3251800,1,22.4,20.2,16.5,25.7,68.3,4.5,9.1,5.5,67.2,67.1,53.8,6.7,29.6,25.0,21.1,54.2,11.3,32.5,2.9,27.8,74.4,15.8,29.0,79.8,14.4,43.1,3.0,18.2,"(36.28919935620, -115.088496088)"
327,328,MI,Ann Arbor,2603000,1,9.0,18.3,23.0,19.1,68.1,4.6,9.8,3.6,66.1,66.4,73.4,4.4,38.6,41.2,13.2,75.5,5.7,29.3,1.9,15.7,84.1,12.3,23.4,83.6,7.9,31.1,1.7,6.6,"(42.27568335670, -83.7311374448)"
416,417,CO,Westminster,883835,1,11.3,20.7,20.7,22.5,66.9,5.5,8.9,4.2,60.6,74.6,65.2,4.3,37.9,34.9,17.2,68.7,6.1,30.6,2.2,15.4,74.5,10.6,24.0,83.5,9.7,29.1,2.1,11.0,"(39.88385114750, -105.062645316)"
306,307,TX,Corpus Christi,4817000,1,27.4,23.9,18.1,32.5,76.7,5.4,8.6,6.5,69.8,72.3,60.4,6.0,31.5,29.8,16.7,56.9,14.4,38.2,3.2,30.0,79.8,12.2,37.2,82.3,14.1,33.9,3.1,15.1,"(27.72915883370, -97.4019095379)"


In [0]:
#Lets remove the geographical variables now
df = df.drop(['Unnamed: 0',
              'Geolocation', 'StateAbbr', 'PlaceName','PlaceFIPS' ], axis =1)

In [0]:
df.shape

(453, 29)

In [0]:
#Lets split the data into training and validation first
y = df['Population2010']
X = df.drop(columns='Population2010')

In [0]:
print(X.shape, y.shape)

(453, 28) (453,)


In [0]:
X.head()

,ACCESS2_CrudePrev,ARTHRITIS_CrudePrev,BINGE_CrudePrev,BPHIGH_CrudePrev,BPMED_CrudePrev,CANCER_CrudePrev,CASTHMA_CrudePrev,CHD_CrudePrev,CHECKUP_CrudePrev,CHOLSCREEN_CrudePrev,COLON_SCREEN_CrudePrev,COPD_CrudePrev,COREM_CrudePrev,COREW_CrudePrev,CSMOKING_CrudePrev,DENTAL_CrudePrev,DIABETES_CrudePrev,HIGHCHOL_CrudePrev,KIDNEY_CrudePrev,LPA_CrudePrev,MAMMOUSE_CrudePrev,MHLTH_CrudePrev,OBESITY_CrudePrev,PAPTEST_CrudePrev,PHLTH_CrudePrev,SLEEP_CrudePrev,STROKE_CrudePrev,TEETHLOST_CrudePrev
229,22.3,25.3,17.3,34.5,77.2,6.4,9.7,6.8,69.8,71.9,60.8,7.2,35.2,29.5,19.8,55.8,12.2,37.7,3.2,28.8,75.3,13.6,36.2,80.8,13.8,35.2,3.7,16.5
337,22.4,20.2,16.5,25.7,68.3,4.5,9.1,5.5,67.2,67.1,53.8,6.7,29.6,25.0,21.1,54.2,11.3,32.5,2.9,27.8,74.4,15.8,29.0,79.8,14.4,43.1,3.0,18.2
327,9.0,18.3,23.0,19.1,68.1,4.6,9.8,3.6,66.1,66.4,73.4,4.4,38.6,41.2,13.2,75.5,5.7,29.3,1.9,15.7,84.1,12.3,23.4,83.6,7.9,31.1,1.7,6.6
416,11.3,20.7,20.7,22.5,66.9,5.5,8.9,4.2,60.6,74.6,65.2,4.3,37.9,34.9,17.2,68.7,6.1,30.6,2.2,15.4,74.5,10.6,24.0,83.5,9.7,29.1,2.1,11.0
306,27.4,23.9,18.1,32.5,76.7,5.4,8.6,6.5,69.8,72.3,60.4,6.0,31.5,29.8,16.7,56.9,14.4,38.2,3.2,30.0,79.8,12.2,37.2,82.3,14.1,33.9,3.1,15.1


In [0]:
y.head()

229    0
337    1
327    1
416    1
306    1
Name: Population2010, dtype: object

In [0]:
validation_size = 0.20
seed = 123
X_train, X_validation, y_train, y_validation = train_test_split(X, y, test_size=validation_size, random_state=seed)

In [0]:
# Spot-Check Algorithms
models = []
models.append(('GBC', GradientBoostingClassifier()))
models.append(('CART', DecisionTreeClassifier()))
models.append(('RFC', RandomForestClassifier()))
models.append(('LDA', LinearDiscriminantAnalysis()))
models.append(('LR', LogisticRegression(max_iter=1000000)))
models.append(('KNN', KNeighborsClassifier()))
models.append(('NB', GaussianNB()))
models.append(('ETC', ExtraTreesClassifier()))
models.append(('BC', BaggingClassifier()))

In [0]:
# evaluate each model in turn
results = []
names = []
# store preds
from sklearn.model_selection import cross_val_predict
dwPreds = []
for name, model in models:
  kfold = KFold(n_splits=20, random_state=seed, shuffle=True)
  cv_results = cross_val_score(model, X_train, y_train, cv=kfold, scoring='accuracy')
  results.append(cv_results)
  names.append(name)
  msg = "%s: %f (%f)" % (name, cv_results.mean(), cv_results.std())
  print(msg)

GBC: 0.635234 (0.100228)
CART: 0.533772 (0.110724)
RFC: 0.579678 (0.150131)
LDA: 0.555702 (0.104777)
LR: 0.547515 (0.104390)
KNN: 0.552339 (0.124277)
NB: 0.599415 (0.097487)
ETC: 0.560819 (0.149213)
BC: 0.518860 (0.153319)


On looking at the results, we can observe that Gradient Boosting Classifier is the best one.

In [0]:
# Make predictions on validation dataset
model1 = GradientBoostingClassifier()
model1.fit(X_train, y_train)
predictions = model1.predict(X_validation)
print(accuracy_score(y_validation, predictions))
print(confusion_matrix(y_validation, predictions))
print(classification_report(y_validation, predictions))

0.4945054945054945
[[24 18]
 [28 21]]
              precision    recall  f1-score   support

           0       0.46      0.57      0.51        42
           1       0.54      0.43      0.48        49

    accuracy                           0.49        91
   macro avg       0.50      0.50      0.49        91
weighted avg       0.50      0.49      0.49        91



In [0]:
# Time for some hyperparameter tuning
#Lets get some information about parameters involved in Gradient boosting
from sklearn.model_selection import GridSearchCV
GradientBoostingClassifier().get_params()

{'ccp_alpha': 0.0,
 'criterion': 'friedman_mse',
 'init': None,
 'learning_rate': 0.1,
 'loss': 'deviance',
 'max_depth': 3,
 'max_features': None,
 'max_leaf_nodes': None,
 'min_impurity_decrease': 0.0,
 'min_impurity_split': None,
 'min_samples_leaf': 1,
 'min_samples_split': 2,
 'min_weight_fraction_leaf': 0.0,
 'n_estimators': 100,
 'n_iter_no_change': None,
 'presort': 'deprecated',
 'random_state': None,
 'subsample': 1.0,
 'tol': 0.0001,
 'validation_fraction': 0.1,
 'verbose': 0,
 'warm_start': False}

In [0]:
#Lets assign a list of parameter values now
param_range1 = [20,30,40]
param_range2 = [4,5,6]
param_range3 = [5,6,7]

In [0]:
grid_parameters = [{
    'max_leaf_nodes': param_range2,
    'max_depth': param_range3,
    'n_estimators': param_range1
 }]


In [0]:
grid = GridSearchCV(GradientBoostingClassifier(), param_grid = grid_parameters, cv = 20, verbose = 5, n_jobs = -1)
grid.fit(X_train, y_train)

Fitting 20 folds for each of 27 candidates, totalling 540 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 2 concurrent workers.
[Parallel(n_jobs=-1)]: Done  24 tasks      | elapsed:    1.1s
[Parallel(n_jobs=-1)]: Done 132 tasks      | elapsed:    8.4s
[Parallel(n_jobs=-1)]: Done 312 tasks      | elapsed:   20.4s
[Parallel(n_jobs=-1)]: Done 540 out of 540 | elapsed:   36.5s finished


GridSearchCV(cv=20, error_score=nan,
             estimator=GradientBoostingClassifier(ccp_alpha=0.0,
                                                  criterion='friedman_mse',
                                                  init=None, learning_rate=0.1,
                                                  loss='deviance', max_depth=3,
                                                  max_features=None,
                                                  max_leaf_nodes=None,
                                                  min_impurity_decrease=0.0,
                                                  min_impurity_split=None,
                                                  min_samples_leaf=1,
                                                  min_samples_split=2,
                                                  min_weight_fraction_leaf=0.0,
                                                  n_estimators=100,
                                                  n_iter_no_change=None,
        

In [0]:
#Lets use some interesting functions that grid search provides us to find out best parameters and score associated with it
grid.best_params_

{'max_depth': 5, 'max_leaf_nodes': 6, 'n_estimators': 40}

In [0]:
grid.best_score_

0.6402046783625731